In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install -q -U watermark

In [ ]:
!pip install -qq transformers

In [ ]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

In [ ]:
#@title Setup & Config
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from transformers import AutoTokenizer, AutoModel

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

## Data Exploration


In [ ]:
df = pd.read_csv("/content/region_detection_train_dataset.csv")
df.head(10)

In [ ]:
df1 = pd.read_csv("/content/region_detection_test_dataset.csv")
df.head(10)

In [ ]:
df2 = pd.read_csv("/content/region_detection_validation_dataset.csv")
df.head(10)

In [ ]:
df=df.dropna()

In [ ]:
df.shape

In [ ]:
df[:]

In [ ]:
df['region_name'].value_counts()

In [ ]:
import pandas as pd

# Create a dictionary to map the class names to integers
class_to_int = {
    "Chittagong": 0,
    "Mymensingh": 4,
    "Noakhali": 1,
    "Sylhet": 2,
    "Barishal": 3
}

# Replace the class names in the DataFrame
df['region_name'] = df['region_name'].replace(class_to_int)

# Print the DataFrame
print(df)

In [ ]:
df['region_name'].value_counts()

In [ ]:
# Create a dictionary to map the class names to integers
# Create a dictionary to map the class names to integers
class_to_int = {
    "Chittagong": 0,
    "Mymensingh": 4,
    "Noakhali": 1,
    "Sylhet ": 2,
    "Barishal": 3
}

In [ ]:
df1['region_name'] = df1['region_name'].replace(class_to_int)

In [ ]:
df1['region_name'].value_counts()

In [ ]:
print(df1)


In [ ]:
import pandas as pd

# Create a dictionary to map the class names to integers
# Create a dictionary to map the class names to integers
class_to_int = {
    "Chittagong": 0,
    "Mymensingh": 4,
    "Noakhali": 1,
    "Sylhet": 2,
    "Barishal": 3
}


In [ ]:
df2['region_name'] = df2['region_name'].replace(class_to_int)

In [ ]:
df2['region_name'].value_counts()

We have about 16k examples. Let's check for missing values:

In [ ]:
df.info()

In [ ]:
#class_names = ['Trans', 'Indigenous','Migrated','Universal Toxic']
class_names = ['Chittagong','Noakhali','Sylhet','Barishal','Mymensingh']

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Increase the figure size
plt.figure(figsize=(10, 6))

Toxic_count = df['region_name'].value_counts()
#sns.countplot(x='Score', data=df, order=Toxic_count.index)
#sns.countplot(x='Score', data=df, order=Toxic_count.index, color='blue')
sns.countplot(x='region_name', data=df, palette=['#FF2400', 'teal','#A52A2A','Seagreen'])
# Rename the x-axis tick labels
new_labels = class_names# Replace with your desired labels
plt.gca().set_xticklabels(new_labels)

plt.show()


In [ ]:
from transformers import AutoTokenizer, AutoModel

bnbert_tokenizer = AutoTokenizer.from_pretrained("sagorsarker/bangla-bert-base")
text = "আমি বাংলায় গান গাই।"
bnbert_tokenizer.tokenize(text)
# ['আমি', 'বাংলা', '##য', 'গান', 'গাই', '।']

from transformers import BertForMaskedLM, BertTokenizer, pipeline

model = BertForMaskedLM.from_pretrained("sagorsarker/bangla-bert-base")
tokenizer = BertTokenizer.from_pretrained("sagorsarker/bangla-bert-base")
nlp = pipeline('fill-mask', model=model, tokenizer=tokenizer)
for pred in nlp(f"আমি বাংলায় {nlp.tokenizer.mask_token} গাই।"):
  print(pred)

# {'sequence': '[CLS] আমি বাংলায গান গাই । [SEP]', 'score': 0.13404667377471924, 'token': 2552, 'token_str': 'গান'}



In [ ]:
from transformers import AutoTokenizer, AutoModel
from transformers import BertForMaskedLM, BertTokenizer, pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-multilingual-cased' #sagorsarker/bangla-bert-base #bert-base-cased #csebuetnlp/banglabert #neuropark/sahajBERT

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME) #AutoTokenizer.from_pretrained("csebuetnlp/banglabert") PreTrainedTokenizerFast.from_pretrained("neuropark/sahajBERT")

In [ ]:
sample_txt = 'আমি মুকাফফি '

In [ ]:
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

In [ ]:
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

In [ ]:
tokenizer.sep_token, tokenizer.sep_token_id

In [ ]:
tokenizer.cls_token, tokenizer.cls_token_id

In [ ]:
tokenizer.pad_token, tokenizer.pad_token_id

In [ ]:
tokenizer.unk_token, tokenizer.unk_token_id

In [ ]:
encoding = tokenizer.encode_plus(
  sample_txt,
  max_length=32,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  truncation=True,
  return_tensors='pt',  # Return PyTorch tensors

)

encoding.keys()

In [ ]:
print(len(encoding['input_ids'][0]))
encoding['input_ids'][0]

In [ ]:
print(len(encoding['attention_mask'][0]))
encoding['attention_mask']

In [ ]:
tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

In [ ]:
token_lens = []

for txt in df.regional_sentence:
  #txt = normalize(txt) # this normalization step is required before tokenizing the text
  tokens = tokenizer.encode(txt, max_length=512)
  token_lens.append(len(tokens))

In [ ]:
sns.distplot(token_lens)
plt.xlim([0, 256]);
plt.xlabel('Token count');

In [ ]:
MAX_LEN = 80

In [ ]:
class GPReviewDataset(Dataset):

  def __init__(self, regional_sentence, targets, tokenizer, max_len):
    self.regional_sentence = regional_sentence
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.regional_sentence)
  def __getitem__(self, item):
    review = str(self.regional_sentence[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'regiona_speech': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [ ]:
df_train, df_test1 = train_test_split(df, test_size=0.0001, shuffle=True)
df_val1, df_test = train_test_split(df1, test_size=0.999,shuffle=False)
df_val,df_test1= train_test_split(df2, test_size=0.0001,shuffle=False)

In [ ]:
df_train.shape, df_val.shape, df_test.shape

In [ ]:

def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = GPReviewDataset(
    regional_sentence=df.regional_sentence.to_numpy(),
    targets=df.region_name.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len,

  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4,
    shuffle=True
  )

In [ ]:
BATCH_SIZE = 8

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

Let's have a look at an example batch from our training data loader:

In [ ]:
data = next(iter(train_data_loader))
data.keys()

In [ ]:
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['targets'].shape)

In [ ]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME) # method name

In [ ]:
!export LRU_CACHE_CAPACITY=1
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

In [ ]:
model.load_state_dict(torch.load('best_model_state.bin'))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
last_hidden_state, pooled_output = bert_model(
  input_ids=encoding['input_ids'],
  attention_mask=encoding['attention_mask']
)

In [ ]:
PRE_TRAINED_MODEL_NAME

In [ ]:
bert_model.config.hidden_size

In [ ]:
class regionClassifier(nn.Module):

  def __init__(self, n_classes):
    super(regionClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME) #model
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
         input_ids=input_ids,
         attention_mask=attention_mask,
         return_dict=False
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
model = regionClassifier(len(class_names))
model = model.to(device)

In [ ]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)

print(input_ids.shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length

In [ ]:
F.softmax(model(input_ids, attention_mask), dim=1)

In [ ]:
#model.load_state_dict(torch.load('best_model_state.bin'))
model = model.to(device)

### Training

In [ ]:
EPOCHS = 10
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=True)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=4,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epoch(
  model,
  data_loader,
  loss_fn,
  optimizer,
  device,
  scheduler,
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0

  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn,
    device,
    len(df_val)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

In [ ]:
model=regionClassifier(len(class_names))

In [ ]:
model.load_state_dict(torch.load('best_model_state.bin'))
model = model.to(device)

## Evaluation


In [ ]:
test_acc, _ = eval_model(
  model,
  test_data_loader,
  loss_fn,
  device,
  len(df_test)
)

test_acc.item()

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()

  regiona_speech = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:

      texts = d["regiona_speech"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      regiona_speech.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return regiona_speech, predictions, prediction_probs, real_values

In [ ]:
y_region_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_data_loader
)

In [ ]:
from sklearn.metrics import log_loss

log_loss_score = log_loss(y_test, y_pred_probs)
print(f'Log Loss: {log_loss_score:.4f}')

Let's have a look at the classification report

In [ ]:
print(classification_report(y_test, y_pred, target_names=class_names,digits=4))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def advanced_confusion_matrix(confusion_matrix, class_names, title):
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(8, 6))

    # Customize the heatmap
    sns.set(font_scale=1.2)  # Adjust font size
    hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues", linewidths=0.5, linecolor="white", cbar=False)

    # Set custom font properties for bold labels
    hmap.set_yticklabels(hmap.get_yticklabels(), rotation=0, ha='right', weight='bold')
    hmap.set_xticklabels(hmap.get_xticklabels(), rotation=0, ha='right', weight='bold')

    # Set axis labels
    plt.ylabel('Actual Region', weight='bold')
    plt.xlabel('Predicted Region', weight='bold')

    # Set title
    plt.title(title, weight='bold')

    plt.show()



In [ ]:
title='Region Detection'

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming you have y_test and y_pred defined
cm = confusion_matrix(y_test, y_pred)
vmin = 0
vmax = 550
xticklabels = ['Chittagong', 'Noakhali', 'Sylhet', 'Barishal', 'Mymensingh']
yticklabels = ['Chittagong', 'Noakhali', 'Sylhet', 'Barishal', 'Mymensingh']

plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt=".0f",
    cmap="Greens",
    vmin=vmin,
    vmax=vmax,
    xticklabels=xticklabels,
    yticklabels=yticklabels
)

font1 = {'size': 20, 'family': 'serif', 'weight': 'bold'}
font2 = {'size': 15, 'family': 'serif'}
plt.title("Bengali Regional Dialects Detection", fontdict=font1, y=1.05)
plt.xlabel("Predicted Region", fontdict=font2)
plt.ylabel("Actual Region", fontdict=font2)
plt.xticks(rotation=90)


# Adjust layout to prevent cropping
plt.tight_layout()

# Save the confusion matrix plot to a PDF file
pdf_filename = 'm_Bertconfusion_matrix_visualization.pdf'
with PdfPages(pdf_filename) as pdf:
    pdf.savefig(plt.gcf())

plt.show()

print(f"Confusion matrix visualization saved to {pdf_filename}")


let's have a look at an example from our test data:

In [ ]:
idx = 50

region_text = y_region_texts[idx]
true_label = y_test[idx]
pred_df = pd.DataFrame({
  'class_names': class_names,
  'values': y_pred_probs[idx]
})

In [ ]:
print("\n".join(wrap(region_text)))
print()
print(f'True Label: {class_names[true_label]}')

Now we can look at the confidence of each sentiment of our model:

In [ ]:
sns.barplot(x='values', y='class_names', data=pred_df, orient='h')
plt.ylabel('Region')
plt.xlabel('probability')
plt.xlim([0, 1]);